<a href="https://colab.research.google.com/github/aaalexlit/tf-advanced-techniques-spec/blob/main/course_1_custom_models/Week3_Activation_in_a_custom_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:

import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dropout, Dense

## Using built-in  Keras activation

Specify `activation` parameter int the `__init()__` method  
Initialize it using `tf.keras.activations.get()`


In [2]:
class SimpleDense(Layer):
  
  # add activation parameter.
  # must be a string identifier of activations available in Keras
  def __init__(self, units=32, activation=None):
    super(SimpleDense, self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)

	# runs when an instance of Layer is created
  # ie defines the initial State
  def build(self, input_shape):
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(name="kernel",
							initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
							trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(name="bias",
							initial_value=b_init(shape=(self.units,), dtype='float32'),
							trainable=True)
	
	# Defines the Computation from inputs to outputs
  # called during training
  def call(self, inputs):
    return self.activation(tf.matmul(inputs, self.w) + self.b)

## Use the layer

In [7]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255., x_test/255.

model = Sequential([
    Flatten(input_shape=(28, 28)),
    SimpleDense(128, activation='relu'),
    # tf.keras.layers.Activation('relu'),
    Dropout(0.2),
    SimpleDense(10, activation='softmax')
])

In [8]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5,
          validation_data=(x_test, y_test))
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3181 - accuracy: 0.9092 - val_loss: 0.1536 - val_accuracy: 0.9550
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1492 - accuracy: 0.9554 - val_loss: 0.1051 - val_accuracy: 0.9668
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1116 - accuracy: 0.9661 - val_loss: 0.0826 - val_accuracy: 0.9752
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0905 - accuracy: 0.9723 - val_loss: 0.0806 - val_accuracy: 0.9748
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0743 - accuracy: 0.9766


[0.07431235164403915, 0.9765999913215637]